## SF crime data analysis and modeling

In [2]:
data source: https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry

In [3]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from ggplot import *
import warnings
from pyspark.sql.functions import to_date, to_timestamp, year, month, dayofmonth, hour, minute
from pyspark.sql.functions import udf, lit
from pyspark.sql.functions import col

import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [4]:
# download data from sf gov
'''import urllib.request
urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))'''

Out[ 2 ]: 'import urllib.request\nurllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")\ndbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")\ndisplay(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))'

In [5]:
# use this file name later
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"

In [6]:
# read data from the data storage
crime_data_lines = sc.textFile(data_path)
#prepare data 
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])
#get header
header = df_crimes.first()
print(header)

#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

#get the total number of data 
print(crimes.count())

['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId', 'SF Find Neighborhoods', 'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods', ':@computed_region_yftq_j783', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py', ':@computed_region_4isq_27mq', ':@computed_region_fcz8_est8', ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7', ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi', ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4']
2215024

### Solove  big data issues via Spark
approach 1: use RDD (not recommend)  
approach 2: use Dataframe, register the RDD to a dataframe
approach 3: use SQL
***note***: you only need to choose one of approaches as introduced above

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.format("csv").option("header", "true").load(data_path)
display(df)
df.createOrReplaceTempView("sf_crime")

IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_yftq_j783,:@computed_region_p5aj_wyqh,:@computed_region_rxqg_mtj9,:@computed_region_bh8s_q3mv,:@computed_region_fyvs_ahh9,:@computed_region_9dfj_4gjx,:@computed_region_n4xg_c4py,:@computed_region_4isq_27mq,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6pnf_4xz7,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4
180362289,VEHICLE THEFT,STOLEN MOTORCYCLE,Tuesday,05/15/2018,10:30,SOUTHERN,NONE,700 Block of TEHAMA ST,-122.41191202732877,37.77520656149669,"(37.77520656149669, -122.41191202732877)",18036228907023,32,1,10,34,8,2,9,28853,34,null,1,null,1,null,null,2,null,null,1,null
180360948,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,05/15/2018,04:14,SOUTHERN,NONE,MARKET ST / SOUTH VAN NESS AV,-122.41925789481357,37.77514629165388,"(37.77514629165388, -122.41925789481357)",18036094864020,32,1,10,20,8,2,9,28853,19,null,1,null,1,null,8,2,1,1,1,null
180360879,OTHER OFFENSES,PAROLE VIOLATION,Tuesday,05/15/2018,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087926150,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null
180360879,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Tuesday,05/15/2018,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087965010,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null
180360879,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,05/15/2018,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087965015,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null
180360829,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Tuesday,05/15/2018,01:27,MISSION,NONE,700 Block of SHOTWELL ST,-122.41561725232026,37.75641376904809,"(37.75641376904809, -122.41561725232026)",18036082965016,53,3,2,20,2,4,7,28859,19,null,null,null,3,null,null,2,null,null,3,null
180360835,ROBBERY,"ROBBERY, BODILY FORCE",Tuesday,05/15/2018,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.41004163181597,37.781953653725715,"(37.781953653725715, -122.41004163181597)",18036083503074,32,5,10,34,14,2,9,28853,34,17,1,18,1,18,7,2,1,1,1,null
180360835,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Tuesday,05/15/2018,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.41004163181597,37.781953653725715,"(37.781953653725715, -122.41004163181597)",18036083516710,32,5,10,34,14,2,9,28853,34,17,1,18,1,18,7,2,1,1,1,null
180360794,LIQUOR LAWS,MISCELLANEOUS LIQOUR LAW VIOLATION,Tuesday,05/15/2018,00:19,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.44776112231955,37.76984648754153,"(37.76984648754153, -122.44776112231955)",18036079417030,25,7,11,3,15,5,11,29492,9,22,null,24,null,25,null,1,null,null,null,null
180360794,WARRANTS,ENROUTE TO OUTSIDE JURISDICTION,Tuesday,05/15/2018,00:19,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.44776112231955,37.76984648754153,"(37.76984648754153, -122.44776112231955)",18036079462050,25,7,11,3,15,5,11,29492,9,22,null,24,null,25,null,1,null,null,null,null


#### Q1 question (OLAP): 
#####Write a Spark program that counts the number of crimes for different category.

Below are some example codes to demonstrate the way to use Spark RDD, DF, and SQL to work with big data. You can follow this example to finish other questions.

In [10]:
# Spark dataframe based
q1_result = df.groupBy('category').count().orderBy('count', ascending=False)
display(q1_result)

category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


In [11]:
#Spark SQL based
crimeCategory = spark.sql("SELECT category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crimeCategory)
# convert result to pandas dataframe
crimes_pd_df = crimeCategory.toPandas()
display(crimes_pd_df)

category,Count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


#### Conclusion for Q1: 
The most frequent type of crime in SF is LARCENY/THEFT.

#### Q2 question (OLAP)
Counts the number of crimes for different district, and visualize your results

In [14]:
# Spark dataframe based
q2_result = df.groupBy('PdDistrict').count().orderBy('count', ascending = False)
display(q2_result)

PdDistrict,count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


In [15]:
# Spark SQL based
crimeDistrict = spark.sql('SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY COUNT DESC')
# convert result to pandas dataframe
district_pd_df = crimeDistrict.toPandas()
display(district_pd_df)

PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


#### Conclusion for Q2: 
1. Crimes occur at the southern district most frequently.
2. It should be interesting to investigate the factors that cause crimes happening in southern district.

#### Q3 question (OLAP)
Count the number of crimes each "Sunday" at "SF downtown".   
hints: SF downtown is defiend  via the range of spatial location. For example, you can use a rectangle to define the SF downtown, or you can define a cicle with center as well. Thus, you need to write your own UDF function to filter data which are located inside certain spatial range. You can follow the example here: https://changhsinlee.com/pyspark-udf/

In [18]:
# Spark SQL based
# According to the map online, I define the center of San Francisco is Union Square and its location is (-122.407437, 37.787994)
# Hence, I think the downtown is a circle area within 0.003 degree from Union Square
downtown_sunday_crime = spark.sql("SELECT Date, DayOfWeek, COUNT(*) AS Count FROM sf_crime WHERE DayOfWeek = 'Sunday' AND \
                   pow(Y - 37.787994, 2) + pow(X + 122.407437, 2) < pow(0.003, 2) GROUP BY Date, DayOfWeek ORDER BY Count DESC")
# convert to dataframe
df_downtown = downtown_sunday_crime.toPandas()
display(df_downtown)

Date,DayOfWeek,Count
03/11/2018,Sunday,30
12/21/2014,Sunday,29
11/25/2007,Sunday,27
12/14/2014,Sunday,27
09/15/2013,Sunday,26
02/12/2006,Sunday,26
01/24/2016,Sunday,25
03/15/2015,Sunday,24
12/18/2016,Sunday,24
08/17/2014,Sunday,23


In [19]:
# Spark dataframe based
# convert the data type of Date from string to date
df = df.withColumn('Date', to_date(df.Date, format = 'MM/dd/yyyy'))
# convert the latitude and longtitue from string to float
df = df .withColumn('Lon', df.X.cast(FloatType())) \
        .withColumn('Lat', df.Y.cast(FloatType()))

In [20]:
# check the df after being changed
display(df)

IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,:@computed_region_yftq_j783,:@computed_region_p5aj_wyqh,:@computed_region_rxqg_mtj9,:@computed_region_bh8s_q3mv,:@computed_region_fyvs_ahh9,:@computed_region_9dfj_4gjx,:@computed_region_n4xg_c4py,:@computed_region_4isq_27mq,:@computed_region_fcz8_est8,:@computed_region_pigm_ib2e,:@computed_region_9jxd_iqea,:@computed_region_6pnf_4xz7,:@computed_region_6ezc_tdp2,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_2dwj_jsy4,Lon,Lat
180362289,VEHICLE THEFT,STOLEN MOTORCYCLE,Tuesday,2018-05-15,10:30,SOUTHERN,NONE,700 Block of TEHAMA ST,-122.41191202732877,37.77520656149669,"(37.77520656149669, -122.41191202732877)",18036228907023,32,1,10,34,8,2,9,28853,34,null,1,null,1,null,null,2,null,null,1,null,-122.41191,37.775208
180360948,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,2018-05-15,04:14,SOUTHERN,NONE,MARKET ST / SOUTH VAN NESS AV,-122.41925789481357,37.77514629165388,"(37.77514629165388, -122.41925789481357)",18036094864020,32,1,10,20,8,2,9,28853,19,null,1,null,1,null,8,2,1,1,1,null,-122.41926,37.775146
180360879,OTHER OFFENSES,PAROLE VIOLATION,Tuesday,2018-05-15,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087926150,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null,-122.417816,37.7571
180360879,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Tuesday,2018-05-15,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087965010,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null,-122.417816,37.7571
180360879,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,2018-05-15,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.41781255878655,37.757100579642824,"(37.757100579642824, -122.41781255878655)",18036087965015,53,3,2,20,2,4,7,28859,19,13,null,15,3,15,null,2,null,null,3,null,-122.417816,37.7571
180360829,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Tuesday,2018-05-15,01:27,MISSION,NONE,700 Block of SHOTWELL ST,-122.41561725232026,37.75641376904809,"(37.75641376904809, -122.41561725232026)",18036082965016,53,3,2,20,2,4,7,28859,19,null,null,null,3,null,null,2,null,null,3,null,-122.41562,37.756413
180360835,ROBBERY,"ROBBERY, BODILY FORCE",Tuesday,2018-05-15,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.41004163181597,37.781953653725715,"(37.781953653725715, -122.41004163181597)",18036083503074,32,5,10,34,14,2,9,28853,34,17,1,18,1,18,7,2,1,1,1,null,-122.41004,37.78195
180360835,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Tuesday,2018-05-15,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.41004163181597,37.781953653725715,"(37.781953653725715, -122.41004163181597)",18036083516710,32,5,10,34,14,2,9,28853,34,17,1,18,1,18,7,2,1,1,1,null,-122.41004,37.78195
180360794,LIQUOR LAWS,MISCELLANEOUS LIQOUR LAW VIOLATION,Tuesday,2018-05-15,00:19,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.44776112231955,37.76984648754153,"(37.76984648754153, -122.44776112231955)",18036079417030,25,7,11,3,15,5,11,29492,9,22,null,24,null,25,null,1,null,null,null,null,-122.44776,37.769848
180360794,WARRANTS,ENROUTE TO OUTSIDE JURISDICTION,Tuesday,2018-05-15,00:19,PARK,"ARREST, BOOKED",1500 Block of HAIGHT ST,-122.44776112231955,37.76984648754153,"(37.76984648754153, -122.44776112231955)",18036079462050,25,7,11,3,15,5,11,29492,9,22,null,24,null,25,null,1,null,null,null,null,-122.44776,37.769848


In [21]:
# the function to compute the distance between the downtown center and a location
def get_distance(lat, lon, lat_center = 37.787994, lon_center = -122.407437):
  distance_to_center = (lat - lat_center)**2 + (lon - lon_center)**2
  return distance_to_center

get_distance_udf = udf(get_distance, FloatType())

In [22]:
# calculate the distance between the location of every crime and downtown center
df = df.withColumn('Distance_to_center', get_distance_udf(df.Lat, df.Lon))

In [23]:
# find out the distance between the location the downtown center is less than 0.003 degree
q3_result = df.filter(df.DayOfWeek == 'Sunday') \
              .filter(df.Distance_to_center <= 0.003**2) \
              .groupBy('Date').count() \
              .orderBy('Date', ascending=False)
display(q3_result)

Date,count
2018-05-13,5
2018-05-06,14
2018-04-29,7
2018-04-22,9
2018-04-15,12
2018-04-08,10
2018-04-01,8
2018-03-25,9
2018-03-18,12
2018-03-11,30


In [24]:
# Since there are no obvious pattern in the downtown crime amount plot,
# I decided to compute the ratio between the crime in downtown and the whole city
sunday_all_count = df.filter(df.DayOfWeek == 'Sunday') \
                     .groupBy('Date').count().withColumnRenamed('count', 'all_count')
# join the table which contains all crimes occurred in Sunday
q3_result = q3_result.join(sunday_all_count, q3_result.Date == sunday_all_count.Date)

In [25]:
# calculate the ratio
q3_result = q3_result.withColumn('Ratio', q3_result['count'] / q3_result['all_count'])
display(q3_result)

Date,count,Date,all_count,Ratio
2005-01-16,16,2005-01-16,365,0.043835616438356165
2006-05-21,2,2006-05-21,358,0.00558659217877095
2009-11-22,6,2009-11-22,288,0.020833333333333332
2004-10-24,9,2004-10-24,371,0.02425876010781671
2006-04-23,13,2006-04-23,371,0.03504043126684636
2009-06-28,3,2009-06-28,382,0.007853403141361256
2009-08-09,12,2009-08-09,346,0.03468208092485549
2009-09-27,4,2009-09-27,358,0.0111731843575419
2011-01-30,6,2011-01-30,313,0.019169329073482427
2003-12-07,6,2003-12-07,289,0.020761245674740483


####Conclusion of Q3:
Based on the result given out above, we can find out the amount of crime occured at downtown in Sunday partly follows a time series pattern. For example, we can see that the amount of crime kept increasing from 2012 to 2015. However, after 2015, the amount of crimes decresed until 2017.

#### Q4 question (OLAP)
Analysis the number of crime in each month of 2015, 2016, 2017, 2018. Then, give your insights for the output results. What is the business impact for your result?

In [28]:
# Spark SQL based
month_crime = spark.sql("SELECT SUBSTRING(Date, 1, 2) AS Month, SUBSTRING(Date, 7) AS Year, COUNT(*) AS Count FROM sf_crime \
                         WHERE CAST(SUBSTRING(Date, 7) AS INT) > 2014 GROUP BY Year, Month ORDER BY Year, Month")
df_month = month_crime.toPandas()
display(df_month)

Month,Year,Count
01,2015,13606
02,2015,12329
03,2015,13929
04,2015,12959
05,2015,13729
06,2015,13304
07,2015,13365
08,2015,13730
09,2015,12896
10,2015,13147


In [29]:
# Spark dataframe based
q4_result = df.filter((year(df.Date) <= 2018) & (year(df.Date) >2014)).groupBy(month(df.Date), year(df.Date)).count()
display(q4_result)

month(Date),year(Date),count
6,2016,12094
1,2017,13084
1,2016,12967
3,2017,13711
11,2016,12720
3,2018,10740
2,2016,12106
6,2017,12605
8,2016,12471
3,2015,13929


####Conclusion of Q4:
According to the result, we can see that crimes occured more frequently at the first several beginning months, except Feb, of each year during 2015 - 2018. I guess this is because the weather in Feb is cold.

#### Q5 question (OLAP)
Analysis the number of crime w.r.t the hour in certian day like 2015/12/15, 2016/12/15, 2017/12/15. Then, give your travel suggestion to visit SF.

In [32]:
# Spark SQL based
# From the result of question 3, I find out crimes happend more frequently at Jan. Hence I pick a certain day from Jan.
hour_crime = spark.sql("SELECT SUBSTRING(Time, 1, 2) AS Hour, COUNT(*) AS Count FROM sf_crime \
                        WHERE SUBSTRING(Date, 1, 5) = '01/15' GROUP BY Hour ORDER BY Hour")
df_hour = hour_crime.toPandas()
display(df_hour)

Hour,Count
00,362
01,186
02,168
03,111
04,94
05,77
06,105
07,172
08,271
09,282


In [33]:
# Spark dataframe based
# I pick another two specific days in this solution
q4_result = df \
              .filter((df.Date == lit('2015-12-15')) | (df.Date == lit('2016-12-15'))) \
              .groupBy(df.Date, hour(df.Time)).count()
display(q4_result)

Date,hour(Time),count
2015-12-15,14,27
2016-12-15,0,22
2015-12-15,5,3
2016-12-15,16,17
2015-12-15,13,13
2015-12-15,3,4
2016-12-15,13,18
2016-12-15,19,31
2015-12-15,8,12
2016-12-15,6,5


####Conclusion of Q5:
According to the result, we can see crimes always happen during afternoon and night(12:00 - 00:00) and crimes happen least frequently at 5am. Thus, visitors should pay more attention when they go out during afternoon.

#### Q6 question (OLAP)
(1) Step1: Find out the top-3 danger disrict  
(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  
(3) give your advice to distribute the police based on your analysis results.

In [36]:
# Spark SQL based
# step 1: find out the top 3 danger districts
top3_danger_dis = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY PdDistrict ORDER BY Count DESC LIMIT 3")
df_top_dis = top3_danger_dis.toPandas()
display(df_top_dis)

PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713


In [37]:
# step 2.1: find out the crime category and time based on the result of step 1
top_category = spark.sql("SELECT PdDistrict, Category, COUNT(*) AS Count FROM sf_crime WHERE PdDistrict \
                          IN ('SOUTHERN', 'MISSION', 'NORTHERN') GROUP BY PdDistrict, Category ORDER BY PdDistrict, Count DESC")
df_top_cate = top_category.toPandas()
display(df_top_cate)

PdDistrict,Category,Count
MISSION,LARCENY/THEFT,48960
MISSION,OTHER OFFENSES,47363
MISSION,NON-CRIMINAL,31911
MISSION,ASSAULT,28791
MISSION,DRUG/NARCOTIC,19251
MISSION,VEHICLE THEFT,17334
MISSION,WARRANTS,16202
MISSION,VANDALISM,14050
MISSION,SUSPICIOUS OCC,10359
MISSION,BURGLARY,9628


In [38]:
# step 2.2: find out the time of crime based on the result of step 1
top_time = spark.sql("SELECT PdDistrict, SUBSTRING(Time, 1, 2) AS Hour, COUNT(*) AS Count FROM sf_crime WHERE PdDistrict \
                      IN ('SOUTHERN', 'MISSION', 'NORTHERN') GROUP BY PdDistrict, Hour ORDER BY PdDistrict, Hour")
df_top_time = top_time.toPandas()
display(df_top_time)

PdDistrict,Hour,Count
MISSION,00,16797
MISSION,01,11125
MISSION,02,8930
MISSION,03,5599
MISSION,04,3920
MISSION,05,3089
MISSION,06,4791
MISSION,07,7449
MISSION,08,10647
MISSION,09,11453


In [39]:
# Spark dataframe based
# step 1: find out the top 3 dangerous district
display(df.groupBy(df.PdDistrict).count().orderBy('count', ascending = False).limit(3))

PdDistrict,count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713


In [40]:
# step 2.1: find out the crime category in these three districts
display( \
       df \
         .filter((df.PdDistrict == 'SOUTHERN') | (df.PdDistrict == 'MISSION') | (df.PdDistrict == 'NORTHERN')) \
         .groupBy(df.Category, df.PdDistrict) \
         .count() \
         .orderBy(df.PdDistrict, 'count', ascending = [True, False]))

Category,PdDistrict,count
LARCENY/THEFT,MISSION,48960
OTHER OFFENSES,MISSION,47363
NON-CRIMINAL,MISSION,31911
ASSAULT,MISSION,28791
DRUG/NARCOTIC,MISSION,19251
VEHICLE THEFT,MISSION,17334
WARRANTS,MISSION,16202
VANDALISM,MISSION,14050
SUSPICIOUS OCC,MISSION,10359
BURGLARY,MISSION,9628


In [41]:
# step 2.2: find out the time of each type of crime in these three districts
display( \
       df \
         .filter((df.PdDistrict == 'SOUTHERN') | (df.PdDistrict == 'MISSION') | (df.PdDistrict == 'NORTHERN')) \
         .groupBy(hour(df.Time), df.PdDistrict) \
         .count() \
         .orderBy(hour(df.Time)))

hour(Time),PdDistrict,count
0,SOUTHERN,20171
0,MISSION,16797
0,NORTHERN,14472
1,SOUTHERN,11203
1,MISSION,11125
1,NORTHERN,9291
2,MISSION,8930
2,NORTHERN,7809
2,SOUTHERN,9295
3,NORTHERN,5450


####Conclusion of Q6:
According to our analysis above, we can see that the three most dangerous district are sourthern, mission and northern. What's more, larceny/theft, other offenses and non-crimial happen most frequently in all of these three districts and most crimes happen during afternoon and night(12:00 - 00:00). Thus, police should patrol more frequently at these districts from afternoon to midnight.

#### Q7 question (OLAP)
For different category of crime, find the percentage of resolution. Based on the output, give your hints to adjust the policy.

In [44]:
# Spark SQL based
# Count the number of each type of resolution for each category
res_count = spark.sql("SELECT Category, Resolution, COUNT(*) AS Count FROM sf_crime \
                       GROUP BY Category, Resolution ORDER BY Category, Count DESC")
res_count.createOrReplaceTempView("res_count")

In [45]:
# Count the number of each category
# From the result above, I define the resolution of NONE and UNFOUNDED as 'unsolved'.
total_count = spark.sql("SELECT Category, Count(*) AS Count FROM sf_crime GROUP BY Category ORDER BY Category ")
total_count.createOrReplaceTempView("total_count")

In [46]:
# Join the two tables above
join_table = spark.sql("SELECT res_count.Category, res_count.Resolution, res_count.Count, total_count.Count AS Total_Count \
                        FROM res_count, total_count \
                        WHERE res_count.Category = total_count.Category")
join_table.createOrReplaceTempView("join_table")

In [47]:
# Get the final result from the join table
result = spark.sql("SELECT Category, (SUM(Count) / Total_count) AS Unsolved_rate \
                    FROM join_table \
                    WHERE Resolution = 'NONE' OR Resolution = 'UNFOUNDED' \
                    GROUP BY Category, Total_count \
                    ORDER BY Unsolved_rate DESC")
result.toPandas()
display(result)
# According to the result, the most resolution many categories of crimes is NONE. I guess NONE means that the record resolution is missed. 

Category,Unsolved_rate
VEHICLE THEFT,0.9539264782546879
RECOVERED VEHICLE,0.9311610830656264
SUSPICIOUS OCC,0.9144125105663568
LARCENY/THEFT,0.9140385640069268
VANDALISM,0.8816808692130726
BURGLARY,0.8432976852408158
BAD CHECKS,0.8172972972972973
ARSON,0.8137878402442127
NON-CRIMINAL,0.8067286833415155
EXTORTION,0.7894736842105263


In [48]:
# Spark dataframe based
# get the unsolved amount for each category
check_unsolved = udf(lambda x: 1 if x in ['NONE', 'UNFOUNDED'] else 0)
q7_result = df.withColumn('Unsolved', check_unsolved(df.Resolution))
q7_result = q7_result \
                  .filter(q7_result.Unsolved == 1) \
                  .groupBy(q7_result.Category) \
                  .count() \
                  .withColumnRenamed('count', 'Unsolved')
# get the total amount for each category
q7_result = q7_result.join( \
                        df.groupBy(df.Category).count() \
                        .withColumnRenamed('count', 'total') \
                        , 'Category')
# get the final result
q7_result = q7_result.withColumn('Unsolved_rate', q7_result.Unsolved / q7_result.total)
display(q7_result.orderBy(q7_result.Unsolved_rate, ascending = False))

Category,Unsolved,total,Unsolved_rate
VEHICLE THEFT,120769,126602,0.9539264782546879
RECOVERED VEHICLE,8116,8716,0.9311610830656264
SUSPICIOUS OCC,73559,80444,0.9144125105663568
LARCENY/THEFT,439148,480448,0.9140385640069268
VANDALISM,102327,116059,0.8816808692130726
BURGLARY,77198,91543,0.8432976852408158
BAD CHECKS,756,925,0.8172972972972973
ARSON,3199,3931,0.8137878402442127
NON-CRIMINAL,192262,238323,0.8067286833415155
EXTORTION,585,741,0.7894736842105263


####Conclusion of Q7:
According to my research, I find out that the resolution rate of some types of crimes is very low, such as vehicle theft. Hence I advise that polices in SF should pay more attention to these types of crimes. For example, they can install some cctv cameras at the districts where crimes always happen.

### Conclusion. 
I applied both of Spark SQL and Spark dataframe to solve all the questions. During this research, I can see that the most dangerous district are Southern, Mission and Northern. What's more, the crimes occur more frequently from noon to midnight during the first several months of a year. Finally, I also foud out that some types of crimes, such as vehicle theft, has a very low resolution rate. Thus, police force in SF should pay more attention to the aspects mentioned above.